In [52]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import time

In [ ]:
LINKS = np.array([])
TOTAL = 0
PAGES = 10

for PAGE in range(1, PAGES + 1):
    TARGET_URL = f'https://www.avito.ma/fr/maroc/voitures?o={PAGE}'
    HEADERS = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"}
    RESPONSE = requests.get(TARGET_URL, headers=HEADERS)
    PAGE_CONTENT = bs(RESPONSE.content, "html.parser")
    LISTINGS = PAGE_CONTENT.find_all("a", class_="sc-1jge648-0")

    for LISTING in LISTINGS:
        LINK = LISTING.get("href") if LISTING and LISTING.get("href") else "N/A"
        LINKS = np.append(LINKS, LINK)
        TOTAL += 1

print(TOTAL)
print(LINKS)

In [ ]:
#Cleaning data
df = pd.Series(LINKS)
df = df.drop_duplicates().reset_index(drop=True)
df.value_counts()
#Array of links
links = np.array(df)

In [ ]:
data = np.array([])
for link in links[2:6]:
    url = link
    header = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"}
    response = requests.get(link, headers = header)
    page = bs(response.content, "html.parser")

    title = page.find("h1", class_="sc-16573058-5 izVEJU")
    title = title.text.strip() if title else None

    price = page.find("div", class_="sc-16573058-10 kRLGQQ")
    price = price.text.strip() if price else None

    adresse_date = page.find_all("span", class_="sc-16573058-17 gLkxLA")
    adresse_date = np.array([data.text.strip() for data in adresse_date]) if adresse_date else []
    adresse = adresse_date[0].split(',')
    city = adresse[1].lstrip(" ") if adresse[1] else None
    district = adresse[0] if adresse[0] else None
    date = adresse_date[1] if adresse_date[0] else None

    owner = page.find("p", class_="sc-1x0vz2r-0 fUTtTl sc-1l0do2b-9 bJuYLD")
    owner = owner.text.strip() if owner else None

    tags = page.find_all("span", class_="sc-1x0vz2r-0 fjZBup")
    tags = np.array([data.text.strip() for data in tags]) if tags else []
    categories = tags[0] if tags[0] else None
    year = tags[1] if tags[1] else None
    transmission = tags[2] if tags[2] else None
    fuel = tags[3] if tags[3] else None
    mileage = tags[4] if tags else None
    make = tags[5] if tags[5] else None
    model = tags[6] if tags[6] else None

    images = page.find_all("img", class_="sc-1gjavk-0 fpXQoT")
    images = np.array([data.get("src") for data in images]) if images else []




title:  Mercedes E300 de AMG FULL FULL 2026
price:  
city : Rabat
district : Yacoub El Mansour
date:  il y a 22 minutes
owner:  Auto Dusseldorf
tags : ["Voitures d'occasion, à vendre" '2026' 'Automatique' 'Hybride' '0'
 'Mercedes-Benz' 'Classe E' 'ABS' 'Airbags' 'CD/MP3/Bluetooth'
 'Caméra de recul' 'Climatisation' 'ESP']
categories : Voitures d'occasion, à vendre
year : 2026
transmission : Automatique
fuel : Hybride
mileage : 0
make : Mercedes-Benz
model : Classe E
images:  ['https://content.avito.ma/classifieds/images/10144780305?t=images'
 'https://content.avito.ma/classifieds/images/10144780307?t=images'
 'https://content.avito.ma/classifieds/images/10144780305?t=images'
 'https://content.avito.ma/classifieds/images/10144780307?t=images']
title:  Volkswagen T-Roc Diesel Automatique 2021
price:  
city : Casablanca
district : Oulfa
date:  il y a 1 heure
owner:  Shaheen Auto
tags : ["Voitures d'occasion, à vendre" '2021' 'Automatique' 'Diesel' '110000'
 'Volkswagen' 'T-Roc' 'ABS' 'Air